In [1]:
#30/7/24 Creates a K fold KNN model for each file in a folder provided they are Fingerprints

import os
import pandas as pd
import numpy as np
import math
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
from Function_Repo import setbalance
import math

seed = 81
#Data import
datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/Fingerprints/'
namelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' and 'organ' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df.drop(columns=['SMILES'], inplace=True)
        df = setbalance(df)
        datasets.append(df)
        namelist.append(filename)

sets = 0
resultslist = []
for df in datasets:
    df = df.dropna()
    dfarray = df.to_numpy()
    
    dfarray, tempset = train_test_split(dfarray, test_size=0.2, 
                                        random_state=seed, stratify = df['Toxicity_Value'])
    
    size = len(dfarray) / 5
    splitsize = math.ceil(size)
    empty = [[] for _ in range(5)]
    splits = []
    for list in empty:
        splits.append(list)
        
    loops = 0
    currsplit = 0
    #Results in a list of 5 lists that each contain 1/5 of the targetdata
    for row in dfarray:
        splits[currsplit].append(row)
        if loops == splitsize:
            loops = 0
            currsplit = currsplit + 1
        loops = loops + 1

    folds = []
    for split in splits:
        temp = pd.DataFrame(split)
        folds.append(temp)
    
    model_list = []
    validlist = []
    totalmetrics = []
    testdat = []
    #For loop that uses each fold once for valid/testing and the rest for training
    #Each 'split' in the range corresponds to the set used for test/validation with the other 4 for training
    iteration = 1
    print('========================================')
    print('For dataset', namelist[sets])
    for split in range(0,5):
        #Set creation
        trainlist = [df for i, df in enumerate(folds) if i != split]
        trainset = pd.concat(trainlist, axis=0)
        testset, validset = train_test_split(folds[split], test_size=0.5, random_state=seed)

        ytrain = trainset[0].values
        xtrain = trainset.iloc[:, 1:]

        ytest = testset[0].values
        xtest = testset.iloc[:, 1:]

        yvalid = validset[0].values
        xvalid = validset.iloc[:, 1:]

        #lists of metric values
        mcclist = []
        bestmetrics = 1
        maxval = 0
        for mtry in range(1, 4):
            #using mtry as the adjusted hyperparameter creates a series of random forests
            rf = SVM = svm.SVC(C=1, kernel='poly', gamma='scale',degree = mtry,
              coef0=0.4, shrinking=True, probability=False, 
              tol=0.001, cache_size=200, class_weight=None, 
              verbose=False, max_iter=-1, decision_function_shape='ovr', 
              break_ties=False, random_state=seed)


            model= rf.fit(xtrain, ytrain)
            preds = model.predict(xtest)
            #calculate metric (mcc)
            mcc = metriccalc(preds, ytest)[12]
            mcclist.append(mcc)
            if mcc > maxval:
                maxval = mcc
                bestmetrics = mtry
        #store best model for the given fold and plot the metric vs mcc value
        rf = SVM = svm.SVC(C=1, kernel='poly', gamma='scale',degree = bestmetrics,
              coef0=0.4, shrinking=True, probability=False, 
              tol=0.001, cache_size=200, class_weight=None, 
              verbose=False, max_iter=-1, decision_function_shape='ovr', 
              break_ties=False, random_state=seed)

        model_list.append(rf)
        testdat.append(maxval)
        totalmetrics.append(mcclist)

        #check models onto validation set, printing various metrics
        model= rf.fit(xtrain, ytrain)
        preds = model.predict(xvalid)
        results = metriccalc(preds, yvalid)
        validlist.append(results)
        print('for fold', split + 1, 'test set mcc of', maxval, 'valid set mcc of', results[12])
        
        iteration = iteration + 1
        
        #get x and y values for the final validation set

    toxvals = []
    fingerprints = []
    for row in tempset:
        toxvals.append(row[0])
        fingerprints.append(row[1:])
        
    #calculates consensus of models on each fingerprint in the final validation set
    consensuslist = []
    predictions = []
    for fp in fingerprints:
        consensus = -1
        fp = fp.reshape(1, -1)
        #appends each model's prediction to a list
        predictions = []
        predlist = []
        for model in model_list:
            preds = model.predict(fp)
            predictions.append(preds)
            predlist.append(preds)
        predictions.append(predlist)

        #finds number of 0s in the prediction list
        zercount = 0
        for num in predictions:
            if num == 0:
                zercount = zercount + 1
        #as there are 5 models, if there are less than 3 0s predicted, the consensus is 1
        if zercount < 3 :
            consensus = 1
        else:
            consensus = 0
        consensuslist.append(consensus)
    #calculate and print metrics
    results = metriccalc(consensuslist, toxvals)
    print('\nvalidation metrics of:')
    print('positives in data', results[0])
    print('negatives in data', results[1])
    print('net accuracy =', results[6])
    print('mcc =',results[12])
    sets += 1
    resultslist.append(results[12])

For dataset SR-ATAD5_Morganfp.csv
for fold 1 test set mcc of 0.5893689584893059 valid set mcc of 0.5262013605584339
for fold 2 test set mcc of 0.5095693779904307 valid set mcc of 0.2720637033616408
for fold 3 test set mcc of 0.5038378351241242 valid set mcc of 0.6690386626313092
for fold 4 test set mcc of 0.26745913993429266 valid set mcc of 0.5335783750799326
for fold 5 test set mcc of 0.2033125151976111 valid set mcc of 0.4930493313022979

validation metrics of:
positives in data 51
negatives in data 53
net accuracy = 0.7884615384615384
mcc = 0.5775064742878283
For dataset NR-AhR_Morganfp.csv
for fold 1 test set mcc of 0.6892889335554799 valid set mcc of 0.4409077231272339
for fold 2 test set mcc of 0.551015445038995 valid set mcc of 0.5048839721464334
for fold 3 test set mcc of 0.6081385697683463 valid set mcc of 0.6957622169940441
for fold 4 test set mcc of 0.6982390321941425 valid set mcc of 0.5778357190665774
for fold 5 test set mcc of 0.6865236565629865 valid set mcc of 0.643956

For dataset Ames_Mutagenicity_Morganfp.csv
for fold 1 test set mcc of 0.5970242459979003 valid set mcc of 0.6166928288813133
for fold 2 test set mcc of 0.6665769895514181 valid set mcc of 0.6582117904715292
for fold 3 test set mcc of 0.6782343603502432 valid set mcc of 0.6176890564613202
for fold 4 test set mcc of 0.6888072125736386 valid set mcc of 0.6479350936814352
for fold 5 test set mcc of 0.6522427262969952 valid set mcc of 0.6402501851447022

validation metrics of:
positives in data 839
negatives in data 658
net accuracy = 0.824315297261189
mcc = 0.6454178938291891
For dataset Respiratory_Toxicity_Morganfp.csv
for fold 1 test set mcc of 0.49071649206571843 valid set mcc of 0.5571406074910352
for fold 2 test set mcc of 0.4732574832468892 valid set mcc of 0.4313388544332112
for fold 3 test set mcc of 0.5720171248332295 valid set mcc of 0.5986171875482424
for fold 4 test set mcc of 0.6484146577066964 valid set mcc of 0.5809040492104167
for fold 5 test set mcc of 0.6032061392810438 

In [2]:
temp = pd.DataFrame(resultslist, columns=['MCC'])
temp2 = pd.DataFrame(namelist, columns=['Dataset'])
resultframe = pd.concat([temp2, temp], axis=1)
resultframe.to_csv('/Users/james/Documents/Honours/Results/Model_Outputs/SVM_Fingerprint_Results.csv')